## Model Tuning for Note Transcription

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow import keras
import keras_tuner as kt
from keras_tuner.tuners import Hyperband
from build_train_set import buildTrainSet
from post_transcription import returnPvLabels
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
x_train, y_train, label_train, x_test, y_test, label_test, x_tune, y_tune, label_tune = buildTrainSet()
print((x_train.shape, y_train.shape), (x_test.shape, y_test.shape), (x_tune.shape, y_tune.shape))

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('input_unit',min_value=32,max_value=512,step=32),
                    return_sequences=True,
                    input_shape=(x_train.shape[1], x_train.shape[2])))
    for i in range(hp.Int('n_layers',1,4)):
        model.add(LSTM(units=hp.Int('lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
    model.add(Dropout(hp.Float('dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(13, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    return model
        

In [ ]:
tuner = Hyperband(
            hypermodel=build_model,
            objective='val_accuracy',
            max_epochs=10,
            factor=3,
            project_name='lstm_tuner')

In [ ]:
tuner.search(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test,y_test))

In [ ]:
tuner.results_summary()